In [ ]:
import psycopg2
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import yaml
import matplotlib.pyplot as plt
from plotnine import *

file = open("../pop.yaml", 'r')
config = yaml.load(file)

engine_path = 'postgresql://{}:{}@{}/{}'
engine = create_engine(engine_path.format(config['db']['username'], config['db']['password'], config['db']['host'],
                                          config['db']['database']))

In [ ]:
#importing required packages -- bokeh, geopandas, shapely, fiona
import requests
import zipfile
import io 


!pip install fiona
import fiona

import geopandas as gp

!pip install shapely 
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

!pip install pysal
import pysal as ps

import matplotlib.pyplot as plt

from bokeh.io import show, output_file, output_notebook, export_png
from bokeh.models import ColumnDataSource, HoverTool, PanTool, WheelZoomTool, LinearColorMapper, BasicTicker, ColorBar, LogColorMapper
from bokeh.palettes import brewer 
from bokeh.plotting import figure, save
from bokeh.resources import CDN
from bokeh.transform import transform
output_notebook()

In [ ]:
#comuna map
url = 'https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10396/1/division_comunal.zip'
request_file = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_file.content))
file_zip.extractall()

In [ ]:
#region map
url = 'https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10398/1/division_regional.zip'
request_file = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_file.content))
file_zip.extractall()

In [ ]:
#urban areas
url = 'https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10405/1/areas_urbanas.zip'
request_file = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_file.content))
file_zip.extractall()

In [ ]:
comunas = gp.read_file('division_comunal.shp')

In [ ]:
regions = gp.read_file('division_regional.shp')

In [ ]:
urban = gp.read_file('areas_urbanas.shp')

In [ ]:
#regions.head()

In [ ]:
#urban.head()

In [ ]:
#c = comunas.plot()

In [ ]:
#comunas.head(10)

In [ ]:
#comunas.loc[comunas['NOM_COM'] == 'Calera'].head(2)

In [ ]:
qry = """
        set role direccion_trabajo_inspections_write;
        SELECT count(IDFiscalizacion) as No_Inspections, empdfcodcomuna_rec, glosa
        FROM cleaned.inspections_se
        group by empdfcodcomuna_rec, glosa;
        """

comuna_inspections =pd.read_sql_query(qry, engine)

#comuna_inspections = comuna_inspections.rename(columns={'empdfcodcomuna_rec': 'COD_COMUNA'})
comuna_inspections = comuna_inspections.rename(columns={'glosa': 'NOM_COM_inspec'})
comuna_inspections['NOM_COM_lower'] = comuna_inspections["NOM_COM_inspec"].map(lambda x: x if type(x)!=str else x.lower())

In [ ]:
comuna_mp = comunas.set_index(['NOM_COM','COD_COMUNA','NOM_REG'])['geometry'].apply(pd.Series).stack().reset_index()

In [ ]:
extract_multi = regions.set_index(['NOM_REG'])['geometry'].apply(pd.Series).stack().reset_index()

In [ ]:
urban_mp = urban.set_index(['NOMBRE'])['geometry'].apply(pd.Series).stack().reset_index()

In [ ]:
comuna_mp['NOM_COM_lower'] = comuna_mp["NOM_COM"].map(lambda x: x.lower())

In [ ]:
#comuna_mp["COD_COMUNA"] = comuna_mp.COD_COMUNA.astype(int)

In [ ]:
#fucntion for extraction of x,y co-ords
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [ ]:
urban_mp['x'] = urban_mp.apply(getPolyCoords, geom=0, coord_type='x', axis=1)
urban_mp['y'] = urban_mp.apply(getPolyCoords, geom=0, coord_type='y', axis=1)

In [ ]:
comuna_mp['x'] = comuna_mp.apply(getPolyCoords, geom=0, coord_type='x', axis=1)
comuna_mp['y'] = comuna_mp.apply(getPolyCoords, geom=0, coord_type='y', axis=1)

In [ ]:
extract_multi['x'] = extract_multi.apply(getPolyCoords, geom=0, coord_type='x', axis=1)
extract_multi['y'] = extract_multi.apply(getPolyCoords, geom=0, coord_type='y', axis=1)

In [ ]:
# removing whitespaces from NOM_COM_lower from inspections database
comuna_inspections['NOM_COM_lower'] = comuna_inspections['NOM_COM_lower'].str.strip()

In [ ]:
comuna_mp = comuna_mp.merge(comuna_inspections, 'left', on='NOM_COM_lower')

In [ ]:
#comuna_mp.head(5)

In [ ]:
#comuna_mp.loc[comuna_mp['NOM_COM_lower'] == 'calera'].head(1)

In [ ]:
#make map
colors = brewer['YlGnBu'][9]
mapper = LinearColorMapper(palette=colors,low=comuna_mp.no_inspections.min(), 
                           high=comuna_mp.no_inspections.max(), nan_color='#525252')

source_1 = ColumnDataSource(data=dict(
    x=comuna_mp['x'], 
    y=comuna_mp['y'], 
    inspections=comuna_mp['no_inspections'],
    comuna_name=comuna_mp['NOM_COM'],
    region=comuna_mp['NOM_REG']
))

source_2 = ColumnDataSource(data=dict(
    x=extract_multi['x'], 
    y=extract_multi['y']
))

source_3 = ColumnDataSource(data=dict(
    x=urban_mp['x'], 
    y=urban_mp['y']
))

c = figure(
    title="No. of Inspections by Comuna in Chile",
    x_axis_location=None, y_axis_location=None,
    plot_width=2500, plot_height=900
)

c.patches('x', 'y', source=source_1, name = 'comuna',
          fill_color=transform('inspections', mapper),
          fill_alpha=0.7, line_color="white", line_width=0.5, legend = 'Comuna')

c.patches('x', 'y', source=source_2,
          fill_alpha=0.0, line_color='black', line_width=1, line_alpha=1, legend = 'Region')

c.patches('x', 'y', source=source_3, line_color='red', fill_alpha=0.7, legend = 'Urban Areas in Chile')
    
#c.circle('x', 'y', source=source_3, size=5, color="navy", alpha=0.5, legend = 'Urban Areas in Chile', 
         #muted_color='navy', muted_alpha=0.2)

c.legend.location = "bottom_right"
c.legend.click_policy="hide"

color_bar = ColorBar(color_mapper=mapper, ticker=BasicTicker(),
                 label_standoff=12, border_line_color=None, location=(0,0),
                 major_label_text_font = 'nunito')

c.add_layout(color_bar, 'right')

#Add tools
hover= HoverTool(tooltips = [
   ("Comuna Name","@comuna_name"),
    ("No. of Inspections","@inspections"),
    ("Region","@region")], names=['comuna'])

c.add_tools(PanTool(), WheelZoomTool(), hover)


In [ ]:
show(c)

In [ ]:
output_file("comuna_urban_region_mp_final.html")

In [ ]:
save(c)